In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data=pd.read_csv('train_final.csv',encoding='Latin-1')
#test_data=pd.read_csv('test_final.csv',encoding='Latin-1')

In [5]:
cols_to_drop=['V300','V309','V111','C3','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V103','V304','V116','V29','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120']

In [6]:
train_data.drop(columns=cols_to_drop,inplace=True)
#test_data.drop(columns=cols_to_drop,inplace=True)

In [7]:
train_data.drop(columns='Unnamed: 0',inplace=True)
#test_data.drop(columns='Unnamed: 0',inplace=True)

In [13]:
train_data

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_36_T,id_37_F,id_37_NaN,id_37_T,id_38_F,id_38_NaN,id_38_T,DeviceType_NaN,DeviceType_desktop,DeviceType_mobile
0,2987000,0,86400,68.50,13926,0.0,150.0,142.0,315.0,87.0,...,0,0,0,1,0,0,1,0,0,1
1,2987001,0,86401,29.00,2755,404.0,150.0,102.0,325.0,87.0,...,0,1,0,0,0,0,1,0,0,1
2,2987002,0,86469,59.00,4663,490.0,150.0,166.0,330.0,87.0,...,0,0,0,1,0,0,1,0,1,0
3,2987003,0,86499,50.00,18132,567.0,150.0,117.0,476.0,87.0,...,0,0,0,1,0,0,1,0,1,0
4,2987004,0,86506,50.00,4497,514.0,150.0,102.0,420.0,87.0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,6550,0.0,150.0,226.0,272.0,87.0,...,0,0,1,0,0,1,0,1,0,0
590536,3577536,0,15811049,39.50,10444,225.0,150.0,224.0,204.0,87.0,...,0,0,1,0,0,1,0,1,0,0
590537,3577537,0,15811079,30.95,12037,595.0,150.0,224.0,231.0,87.0,...,0,0,1,0,0,1,0,1,0,0
590538,3577538,0,15811088,117.00,7826,481.0,150.0,224.0,387.0,87.0,...,0,0,1,0,0,1,0,1,0,0


In [8]:
features=['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'P_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M5', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13','V17', 'V19', 'V20', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140', 'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177', 'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V291', 'V292', 'V294', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V317', 'V322', 'V323', 'V324', 'V326', 'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09', 'id_11', 'id_13', 'id_14', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33', 'DeviceInfo', 'device_name', 'device_version', 'OS_id_30', 'version_id_30', 'navegador_id_31', 'version_id_31', 'resolucion_H', 'resolucion_V', 'ProductCD_C', 'ProductCD_H', 'ProductCD_R', 'ProductCD_S', 'ProductCD_W', 'card4_NaN', 'card4_american express', 'card4_discover', 'card4_mastercard', 'card4_visa', 'card6_NaN', 'card6_charge card', 'card6_credit', 'card6_debit', 'card6_debit or credit', 'M1_F', 'M1_NaN', 'M1_T', 'M2_F', 'M2_NaN', 'M2_T', 'M3_F', 'M3_NaN', 'M3_T', 'M4_M0', 'M4_M1', 'M4_M2', 'M4_NaN', 'M6_F', 'M6_NaN', 'M6_T', 'M7_F', 'M7_NaN', 'M7_T', 'M8_F', 'M8_NaN', 'M8_T', 'M9_F', 'M9_NaN', 'M9_T', 'id_12_Found', 'id_12_NaN', 'id_12_NotFound', 'id_15_Found', 'id_15_NaN', 'id_15_New', 'id_15_Unknown', 'id_16_Found', 'id_16_NaN', 'id_16_NotFound', 'id_28_Found', 'id_28_NaN', 'id_28_New', 'id_29_Found', 'id_29_NaN', 'id_29_NotFound', 'id_34_NaN', 'id_34_match_status:-1', 'id_34_match_status:0', 'id_34_match_status:1', 'id_34_match_status:2', 'id_35_F', 'id_35_NaN', 'id_35_T', 'id_36_F', 'id_36_NaN', 'id_36_T', 'id_37_F', 'id_37_NaN', 'id_37_T', 'id_38_F', 'id_38_NaN', 'id_38_T', 'DeviceType_NaN', 'DeviceType_desktop', 'DeviceType_mobile']

In [5]:
print(list(train_data.columns))

['Unnamed: 0', 'TransactionID_x', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M5', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V

In [16]:
print(features)

[['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'P_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M5', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17', 'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140', 'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V17

In [8]:
X = train_data[features]
Y = train_data['isFraud']

X_train,X_test,Y_train,Y_test = train_test_split(X,Y)

In [9]:
del train_data,

In [10]:
from sklearn import preprocessing
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))  
        

print('Labelling done.')

Labelling done.


In [11]:
from sklearn.model_selection import TimeSeriesSplit,KFold
n_fold = 3
folds = KFold(n_splits=n_fold,shuffle=True)

print(folds)

KFold(n_splits=3, random_state=None, shuffle=True)


In [22]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

xgbclf = xgb.XGBClassifier(
        n_estimators=50,
        max_depth=3,
        learning_rate=0.048,
        subsample=0.85,
        colsample_bytree=0.85,
        missing=-999,
        reg_alpha=0.15,
        reg_lamdba=0.85
    )


In [23]:
X_train_, X_valid = X_train.iloc[train_index], X_train.iloc[valid_index]
Y_train_, Y_valid = Y_train.iloc[train_index], Y_train.iloc[valid_index]
a = xgbclf.fit(X_train_,Y_train_)    

NameError: name 'X_train' is not defined

In [14]:
del X_train, Y_train

In [15]:
test_data=pd.read_csv('test_final.csv',encoding='Latin-1')

In [73]:
test_data.drop(columns=cols_to_drop,inplace=True)
test_data.drop(columns='Unnamed: 0',inplace=True)

In [17]:
for f in test.columns:
    if test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test[f].values))
        test[f] = lbl.transform(list(test[f].values))
        

print('Labelling done.')

Labelling done.


In [16]:
test=test_data[features]

In [19]:
pred=xgbclf.predict_proba(test)[:,1]
val=xgbclf.predict_proba(X_valid)[:,1]
del xgbclf, X_valid
print('ROC accuracy: {}'.format(roc_auc_score(Y_valid, val)))
del val,Y_valid


NameError: name 'xgbclf' is not defined

In [ ]:
xgb['TransactionID']=test_data['TransactionID_x']
xgb['isFraud'] = xgb_submission['isFraud']+pred/n_fold
del pred

In [77]:
print(list(test_data))

['TransactionID_x', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M5', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96'

In [ ]:
pred=xgbclf.predict_proba(X_test)[:,1]
val=xgbclf.predict_proba(X_valid)[:,1]
del xgbclf, X_valid
print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
del val,y_valid
xgb_submission['isFraud'] = xgb_submission['isFraud']+pred/n_fold
del pred

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train_data=reduce_mem_usage(train_data)

Mem. usage decreased to 605.42 Mb (73.0% reduction)


In [5]:
features = ['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'M5', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V107', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_30', 'id_31', 'id_32', 'id_33', 'DeviceInfo', 'OS_id_30', 'version_id_30', 'version_id_31', 'device_name', 'device_version', 'navegador_id_31', 'resolucion_H', 'resolucion_V', 'ProductCD_C', 'ProductCD_H', 'ProductCD_R', 'ProductCD_S', 'ProductCD_W', 'card4_NaN', 'card4_american express', 'card4_discover', 'card4_mastercard', 'card4_visa', 'card6_NaN', 'card6_charge card', 'card6_credit', 'card6_debit', 'M1_F', 'M1_NaN', 'M1_T', 'M2_F', 'M2_NaN', 'M2_T', 'M3_F', 'M3_NaN', 'M3_T', 'M4_M0', 'M4_M1', 'M4_M2', 'M4_NaN','M8_F', 'M8_NaN', 'M8_T', 'M9_F', 'M9_NaN', 'M9_T', 'id_12_Found', 'id_12_NaN', 'id_12_NotFound', 'id_15_Found', 'id_15_NaN', 'id_15_New', 'id_15_Unknown', 'id_16_Found', 'id_16_NaN', 'id_16_NotFound', 'id_28_Found', 'id_28_NaN', 'id_28_New', 'id_29_Found', 'id_29_NaN', 'id_29_NotFound', 'id_35_F', 'id_35_NaN', 'id_35_T', 'id_36_F', 'id_36_NaN', 'id_36_T', 'id_37_F', 'id_37_NaN', 'id_37_T', 'DeviceType_NaN', 'DeviceType_desktop', 'DeviceType_mobile']

In [ ]:
te